#### Spotify API:
- API connection
- Data sorting
- Data to CSV

In [ ]:
# API connection

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


CLIENT_ID = '3f8cc110d1534d4ba1cd6d063cf114e3'
CLIENT_SECRET = '2c63ffda0e7d4247aaa719de974460bf'


auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Data sorting

spotify_data = []

def data_extraction_s(list_url):  # pongo la url de la playlist de spotify en la funcion, cada url es una playlist de un año.

    for url in list_url:  # recorre cada url y saca informacion

        playlist = sp.playlist(url, fields=None, market=None, additional_types= 'track') # descarga informacion completa de la playlist por separado 

        
        for item in playlist["tracks"]["items"]:  # itera por cada pist de la playlist y saca track, artist y año
            track = item["track"]
            artist = item["track"]["artists"]
            eurovisionyear = playlist['name']

            spotify_data.append({     # guarda los datos que le pido en un diccionario
                "Artist": artist[0]["name"].title(),
                "Track": track["name"].title(),
                "Spotify_Duration": track["duration_ms"],
                "Spotify_EurovisionYear" : eurovisionyear.title(),
                "Spotify_Popularity": track["popularity"]
                })
        
        

In [ ]:
# Data to CSV: convierto la info en un csv

raw_data_s = data_extraction_s([
    "https://open.spotify.com/playlist/3ZdQUt8Tmtt7oOU8UM2koe?si=eea27348cf654d21", 
    "https://open.spotify.com/playlist/5h0sQpJnLVzgy5iOH1UNcl?si=3a1d926528484506", 
    "https://open.spotify.com/playlist/3EBtzzGyrlDUr1QvXVGz0h?si=DMNkLRIjQE6THtHUYc-oQw",
    "https://open.spotify.com/playlist/61fyG82H3Cha91cgKgQT6S?si=o-Xs3DfHRZyRuR8qaY9mxAW",
    "https://open.spotify.com/playlist/2HMW5JNZNh9HLfIHNzY0DP?si=f29ed55294394682"
    ])

df_spotify = pd.DataFrame(spotify_data)
df_spotify

df_spotify.to_csv('Spotify_data.csv')


In [ ]:
df_spotify

#### Eurovision API:
- Api connection & Sorting
- Data to CSV

In [ ]:
import requests
import pandas as pd
import json

In [ ]:
# API connection & Sorting

eurovision_data = []
def data_extraction_e(list_years):

    eu_data = requests.get(f"https://eurovisionapi.runasp.net/api/contests/{list_years}/")

    eu_data_json = eu_data.json()
    
    if 'contestants' in eu_data_json:
        
        for idx, i in enumerate(eu_data_json["contestants"], start=1):

            contestant_name = i["artist"]
            song_name = i["song"]
            country = i["country"]

            eurovision_data.append({
                "Ranking": idx,
                "Artist": contestant_name.title(),
                "Track": song_name.title(),
                "Country": country.upper(),
                "Eurovision_Year": list_years
                })

In [ ]:
# Data to CSV

eurovision_years= [2019,2021,2022,2023,2024]

for year in eurovision_years:
    raw_data_e = data_extraction_e(year)
df_eurovision = pd.DataFrame(eurovision_data)
df_eurovision

df_eurovision.to_csv('Eurovision_data.csv', index=False)


#### LastFM API:


- Artists

In [ ]:
import pandas as pd
import requests
import json
import re  # Importar el módulo para limpiar HTML

In [ ]:
# FUNCION PARA INFO DE UN ARTISTA:

api_key = "aedf9c3eda73abe2089afdb473b10063"
share_secret = "b8cd0d2ece4bd3f9f8abb2b3b6d72c6d"

def obtener_info_artista(artist_name, api_key):  

    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={api_key}&format=json"
    
    response = requests.get(url)  
    info_artist = response.json() 

  
    if "error" in info_artist:
        print(f"Error en la API: {info_artist['message']}")
        return pd.DataFrame()  

    
    nombre = info_artist["artist"].get("name")
    url = info_artist["artist"].get("url") 
    oyentes = info_artist["artist"]["stats"].get("listeners")
    reproducciones = info_artist["artist"]["stats"].get("playcount")

    bio_summary = info_artist["artist"]["bio"].get("summary")
    bio_summary_clean = re.sub(r"<.*?>", "", bio_summary)  # Eliminar etiquetas HTML

    generos = [tag["name"] for tag in info_artist["artist"]["tags"]["tag"][:3]] 
    generos_texto = ", ".join(generos)  # utilizo este codigo para que aparezca como cadena de texto y no como lista con corchetes y comillas.

    artistas_similares = [artista["name"] for artista in info_artist["artist"]["similar"]["artist"][:3]]  
    artistas_similares_texto = ", ".join(artistas_similares)   # igual que en genero lo convierto en cadena de texto.

    return {                    
        "artista": nombre,
        "url":url,
        "oyentes": oyentes,
        "reproducciones": reproducciones,
        "generos": generos_texto,
        "artistas_similares": artistas_similares_texto,
        "biografia_resumen": bio_summary_clean
    }



In [ ]:
#lista artistas por año:

popular_artists_2019 = ["Duncan Laurence", "KEiiNO","MARUV", "Tamta","Kate Miller-Heidke" ]
popular_artists_2021 = ["Barbara Pravi", "Måneskin","Efendi", "Tusse", "THE ROOP"]
popular_artists_2022 = ["Mahmood", "LUM!X", "S10", "MARO", "Subwoolfer"]
popular_artists_2023 = ["Loreen", "Alessandra","Käärijä", "Marco Mengoni", "BLANKA"]
popular_artists_2024 = ["Joost", "Nemo", "Slimane", "Angelina Mango", "Baby Lasagna"]

artists_per_years = [
    popular_artists_2019,
    popular_artists_2021,
    popular_artists_2022,
    popular_artists_2023,
    popular_artists_2024]

years =[2019, 2021, 2022, 2023, 2024]

def get_info_artists_per_year(artists_per_years, api_key, years):
    data_artists = []  
    
    for artists, year in zip(artists_per_years, years): # ZIP permite combinar dos o mas listas de manera que puedas iterar sobre ellas al mismo tiempo.
        for artist in artists:
            info = obtener_info_artista(artist, api_key)  
            if info: 
                info["year"] = year 
                data_artists.append(info)  
    
    
    df_artists = pd.DataFrame(data_artists)
    return df_artists

In [ ]:
df_artists = get_info_artists_per_year(artists_per_years, api_key, years)
print(df_artists.head())

In [ ]:
df_artists.to_csv('LastFM_25_Artists_Info.csv', index=False)
print(df_artists)

In [ ]:
from IPython.display import display

display(df_artists)

#### MySQL Data Base
- MySQL connection
- Schema Creation and Use
- Tables Creation

In [ ]:
# MySQL connection

import mysql.connector

try:
    cnx = mysql.connector.connect(user='root', password='root',
                              host='127.0.0.1')
    

    mycursor = cnx.cursor()
    

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

In [139]:
# Schema Creation and Use

query_create_schema = """CREATE SCHEMA StreamMusic"""
mycursor.execute(query_create_schema)
mycursor.execute('USE StreamMusic')

In [ ]:
# Tables creation


the_tables = {  'API_Eurovision' :          """CREATE TABLE api_eurovision (
                                                Ranking INT NOT NULL,
                                                Artist VARCHAR(45) NOT NULL,                              
                                                Track VARCHAR(70) NOT NULL,
                                                Country VARCHAR(3) NOT NULL,
                                                Eurovision_Year YEAR NOT NULL)""",
                                                
                'API_Spotify_Eurovision' :    """CREATE TABLE api_spotify_merged_eurovision (
                                                Artist VARCHAR(45),
                                                Track VARCHAR(70),
                                                Spotify_Duration INT,
                                                Spotify_EurovisionYear YEAR,
                                                Spotify_Popularity VARCHAR(10),
                                                PRIMARY KEY (Artist, Track)
                                                )""", 
                
                'API_LastFM_Info_Artists' :  """CREATE TABLE api_lastfm_info_artists (
                                                Artist VARCHAR(45),
                                                Listeners INT,
                                                Playcount INT,
                                                Genre1 VARCHAR(45),
                                                Genre2 VARCHAR(45),
                                                Genre3 VARCHAR(45),
                                                Similar_Artists1 VARCHAR(45),
                                                Similar_Artists2 VARCHAR(45),
                                                Similar_Artists3 VARCHAR(45),
                                                PRIMARY KEY (Artist),
                                                CONSTRAINT fk_artists_from_s
                                                        FOREIGN KEY (Artist)
                                                        REFERENCES  api_spotify_merged_eurovision (Artist)
                                                        ON UPDATE CASCADE
                                                        ON DELETE CASCADE
                                                )""",
                
                'API_LastFM_Top_Tracks' :    """CREATE TABLE api_lastfm_top_tracks (
                                                Artist VARCHAR(45) NOT NULL,
                                                Release_year YEAR NOT NULL,
                                                Top_track VARCHAR(150),
                                                PRIMARY KEY (Artist),
                                                CONSTRAINT fk_tracks_from_s
                                                        FOREIGN KEY (Artist)
                                                        REFERENCES  api_spotify_merged_eurovision (Artist)
                                                        ON UPDATE CASCADE
                                                        ON DELETE CASCADE
                                                )""" 
                }



for k, v in the_tables.items():
        print(f'Table created: {k}')
        mycursor.execute(v)

        print('------------')



Table created: API_Eurovision
------------
Table created: API_Spotify_Eurovision
------------
Table created: API_LastFM_Info_Artists
------------
Table created: API_LastFM_Top_Tracks
------------


#### Data Import

- Method
- Queries
- Data Import

In [141]:
# Data Import Method

def data_import(query,values):

    mycursor = cnx.cursor()
    mycursor.executemany(query, values)
    print(mycursor.rowcount,'values entered')
    cnx.commit()
 

In [142]:
# Data Import Queries 
query_e_import =            """ INSERT INTO API_EUROVISION (Ranking, Artist, Track, Country, Eurovision_Year)
                                VALUES (%s, %s, %s, %s, %s) """

query_s_e_import =            """ INSERT INTO api_spotify_merged_eurovision (Artist, Track, Spotify_Duration, Spotify_EurovisionYear, Spotify_Popularity)
                                VALUES (%s, %s, %s, %s, %s) """

query_l_artist_import =     """ INSERT INTO api_lastfm_info_artists (Artist, Listeners, Playcount, Genre1, Genre2, Genre3, Similar_Artists1, Similar_Artists2, Similar_Artists3)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) """

query_l_toptracks_import =  """ INSERT INTO api_lastfm_top_tracks (Release_year, Artist, Top_track)
                                VALUES (%s, %s, %s) """



In [143]:
# Eurovision Data Import

e_file = pd.read_csv('clean_csv/Eurovision_data_clean.csv', sep=";")

e_file

,Ranking,Artist,Track,Country,Eurovision_Year
0,30,Anna Odobescu,Stay,MD,2019
1,16,Bilal Hassani,Roi,FR,2019
2,36,Carousel,That Night,LV,2019
3,8,Chingiz,Truth,AZ,2019
4,35,Conan Osiris,Telemoveis,PT,2019
...,...,...,...,...,...
189,14,Silvester Belt,Luktelk,LT,2024
190,4,Slimane,Mon Amour,FR,2024
191,13,Tali,Fighter,LU,2024
192,17,Teya Dora,Ramonda,RS,2024


In [144]:
e_values = list(e_file.itertuples(index=False, name=None))


In [145]:
eurovisionimp = data_import(query_e_import, e_values)

194 values entered


In [146]:
# Spotify Data Import

s_file = pd.read_csv('clean_csv/Spotify_data_clean.csv', sep=";")
s_file



,Column1,Artist,Track,Spotify_Duration,Spotify_EurovisionYear,Spotify_Popularity
0,33,Anna Odobescu,Stay,180517,2019,19
1,9,Bilal Hassani,Roi,177250,2019,21
2,34,Carousel,That Night,181523,2019,25
3,0,Chingiz,Truth,209411,2019,0
4,39,Conan Osiris,Telemoveis,185688,2019,29
...,...,...,...,...,...,...
190,185,Silvester Belt,Luktelk,161142,2024,0
191,176,Slimane,Mon Amour,180908,2024,64
192,192,Tali,Fighter,179738,2024,50
193,197,Teya Dora,Ramonda,174352,2024,0


In [148]:
s_file = s_file.iloc[:, 1:]
s_values = list(s_file.itertuples(index = False, name=None))
spotifyimp = data_import(query_s_e_import, s_values) 

195 values entered


In [ ]:
# LastFM Artists Import

# l_file = pd.read_csv(,index_col=0)

